https://www.kaggle.com/asindico/a-japanese-journey
https://www.kaggle.com/timolee/feeling-hungry-a-beginner-s-guide-to-arima-models

In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%html
    <style>.container { width:90% !important;} </style>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
import calendar

/usr/local/lib/python2.7/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
import calendar
#
air = pd.read_csv('../input/air_visit_data.csv', parse_dates=[1])
air.set_index(['visit_date'],inplace=True)
air.index.name=None
air.drop('air_store_id',axis=1,inplace=True)
df2=pd.DataFrame()
df2['visit_total'] = air.groupby(air.index,squeeze=True,sort=True)['visitors'].sum()
df2['visit_mean'] = air.groupby(air.index,squeeze=True,sort=True)['visitors'].mean()
df2['reserv_cnt'] = air.groupby(air.index,squeeze=True,sort=True)['visitors'].count()
air=df2;del df2

#Get the date info with dow and holidays
hol=pd.read_csv('../input/date_info.csv', parse_dates=True).rename(columns={'calendar_date':'visit_date'})
hol['visit_date'] = pd.to_datetime(hol['visit_date'])
hol.set_index(['visit_date'], inplace=True)
hol.index.name=None
hol.day_of_week = hol.day_of_week.apply(list(calendar.day_name).index)

#Get the test submission
test = pd.read_csv('../input/sample_submission.csv')
test['store_id'], test['visit_date'] = test['id'].str[:20], test['id'].str[21:]
test.set_index('visit_date', drop=True, inplace=True)
test.index.name=None

In [5]:
airreserv = pd.read_csv('../input/air_reserve.csv')
airstore = pd.read_csv('../input/air_store_info.csv')
hpg = pd.read_csv('../input/hpg_reserve.csv')
hpgstore = pd.read_csv('../input/hpg_store_info.csv')

airreserv = pd.merge(airreserv,airstore,on='air_store_id')
hpg = pd.merge(hpg,hpgstore,on='hpg_store_id')
rel = pd.read_csv('../input/store_id_relation.csv')
airrel = pd.merge(airreserv,rel,how='left',on='air_store_id')
hpgrel = pd.merge(hpg,rel,how='left',on='hpg_store_id')
full = pd.merge(airrel,hpgrel,how='outer')
print("there are ",len(air)," restaurants with AIR and ",len(hpg)," with HPG.",len(rel),' have both.')

air.head()
#airreserv.head()
full.head()

('there are ', 478, ' restaurants with AIR and ', 871308, ' with HPG.', 150, ' have both.')


,visit_total,visit_mean,reserv_cnt
2016-01-01,1033,21.520833,48
2016-01-02,1764,28.000000,63
2016-01-03,2368,29.234568,81
2016-01-04,3326,21.184713,157
2016-01-05,3927,17.000000,231


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_genre_name,air_area_name,latitude,longitude,hpg_store_id,hpg_genre_name,hpg_area_name
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN


In [11]:
air_genre = full.loc[full.air_genre_name.isnull()==False].groupby(['air_genre_name'],as_index=False).count()
hpg_genre = full.loc[full.hpg_genre_name.isnull()==False].groupby(['hpg_genre_name'],as_index=False).count()


air_genre.head()
hpg_genre.head()
genres = airreserv.air_genre_name.unique()
genres

,air_genre_name,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_area_name,latitude,longitude,hpg_store_id,hpg_genre_name,hpg_area_name
0,Asian,2,2,2,2,2,2,2,0,0,0
1,Bar/Cocktail,910,910,910,910,910,910,910,724,0,0
2,Cafe/Sweets,3862,3862,3862,3862,3862,3862,3862,606,0,0
3,Creative cuisine,1270,1270,1270,1270,1270,1270,1270,545,0,0
4,Dining bar,8243,8243,8243,8243,8243,8243,8243,4178,0,0


,hpg_genre_name,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_genre_name,air_area_name,latitude,longitude,hpg_store_id,hpg_area_name
0,Amusement bar,168,415,415,415,0,0,415,415,415,415
1,Bar/Cocktail,0,881,881,881,0,0,881,881,881,881
2,Bistro,0,4181,4181,4181,0,0,4181,4181,4181,4181
3,Cafe,213,4435,4435,4435,0,0,4435,4435,4435,4435
4,Cantonese food,0,642,642,642,0,0,642,642,642,642


array(['Japanese food', 'Dining bar', 'Izakaya',
       'Okonomiyaki/Monja/Teppanyaki', 'Italian/French', 'Cafe/Sweets',
       'Yakiniku/Korean food', 'Western food', 'Bar/Cocktail', 'Other',
       'Creative cuisine', 'Karaoke/Party', 'International cuisine',
       'Asian'], dtype=object)

In [8]:
#df2['visit_total'] = air.groupby(air.index,squeeze=True,sort=True)['visitors'].sum()
df=full[full.columns[0:8]]
#df.set_index('visit_datetime', inplace=True)

grp=df.groupby(['visit_datetime','air_genre_name'],squeeze=False,sort=True)['reserve_visitors'].sum()
#grp.index.name=None
df2=pd.DataFrame(grp)

df2.head()
df2.index

reserve_visitors
visit_datetime      air_genre_name                  
2016-01-01 19:00:00 Dining bar                     9
                    Japanese food                  1
2016-01-01 20:00:00 Dining bar                     5
                    Japanese food                  2
2016-01-02 01:00:00 Dining bar                     6

MultiIndex(levels=[[u'2016-01-01 13:00:00', u'2016-01-01 17:00:00', u'2016-01-01 18:00:00', u'2016-01-01 19:00:00', u'2016-01-01 20:00:00', u'2016-01-01 21:00:00', u'2016-01-01 22:00:00', u'2016-01-02 01:00:00', u'2016-01-02 11:00:00', u'2016-01-02 12:00:00', u'2016-01-02 13:00:00', u'2016-01-02 14:00:00', u'2016-01-02 16:00:00', u'2016-01-02 17:00:00', u'2016-01-02 18:00:00', u'2016-01-02 19:00:00', u'2016-01-02 20:00:00', u'2016-01-02 21:00:00', u'2016-01-02 22:00:00', u'2016-01-02 23:00:00', u'2016-01-03 00:00:00', u'2016-01-03 01:00:00', u'2016-01-03 11:00:00', u'2016-01-03 12:00:00', u'2016-01-03 13:00:00', u'2016-01-03 14:00:00', u'2016-01-03 15:00:00', u'2016-01-03 16:00:00', u'2016-01-03 17:00:00', u'2016-01-03 18:00:00', u'2016-01-03 19:00:00', u'2016-01-03 20:00:00', u'2016-01-03 21:00:00', u'2016-01-03 22:00:00', u'2016-01-04 11:00:00', u'2016-01-04 12:00:00', u'2016-01-04 13:00:00', u'2016-01-04 14:00:00', u'2016-01-04 15:00:00', u'2016-01-04 16:00:00', u'2016-01-04 17:00:0

In [16]:
grp
grp.index

visit_datetime       air_genre_name              
2016-01-01 19:00:00  Dining bar                       9
                     Japanese food                    1
2016-01-01 20:00:00  Dining bar                       5
                     Japanese food                    2
2016-01-02 01:00:00  Dining bar                       6
2016-01-02 16:00:00  Izakaya                          4
2016-01-02 17:00:00  Italian/French                   8
                     Izakaya                         67
                     Okonomiyaki/Monja/Teppanyaki     2
2016-01-02 18:00:00  Dining bar                       5
                     Italian/French                   3
                     Japanese food                    4
                     Okonomiyaki/Monja/Teppanyaki     2
2016-01-02 19:00:00  Dining bar                       8
                     Okonomiyaki/Monja/Teppanyaki     2
2016-01-02 21:00:00  Dining bar                      15
                     Japanese food                    

MultiIndex(levels=[[u'2016-01-01 13:00:00', u'2016-01-01 17:00:00', u'2016-01-01 18:00:00', u'2016-01-01 19:00:00', u'2016-01-01 20:00:00', u'2016-01-01 21:00:00', u'2016-01-01 22:00:00', u'2016-01-02 01:00:00', u'2016-01-02 11:00:00', u'2016-01-02 12:00:00', u'2016-01-02 13:00:00', u'2016-01-02 14:00:00', u'2016-01-02 16:00:00', u'2016-01-02 17:00:00', u'2016-01-02 18:00:00', u'2016-01-02 19:00:00', u'2016-01-02 20:00:00', u'2016-01-02 21:00:00', u'2016-01-02 22:00:00', u'2016-01-02 23:00:00', u'2016-01-03 00:00:00', u'2016-01-03 01:00:00', u'2016-01-03 11:00:00', u'2016-01-03 12:00:00', u'2016-01-03 13:00:00', u'2016-01-03 14:00:00', u'2016-01-03 15:00:00', u'2016-01-03 16:00:00', u'2016-01-03 17:00:00', u'2016-01-03 18:00:00', u'2016-01-03 19:00:00', u'2016-01-03 20:00:00', u'2016-01-03 21:00:00', u'2016-01-03 22:00:00', u'2016-01-04 11:00:00', u'2016-01-04 12:00:00', u'2016-01-04 13:00:00', u'2016-01-04 14:00:00', u'2016-01-04 15:00:00', u'2016-01-04 16:00:00', u'2016-01-04 17:00:0